In [1]:
import ROOT as r
from math import *
import array
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, '../include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE

Welcome to JupyROOT 6.26/06


In [5]:
pas="1"
config="3"
mup=2.793

In [6]:
Q2=CONFIG.Function_JSON("Q2",f"../config/cuts{config}.cfg")

<span style="font-size:1.5em;">
\begin{equation}
A_{\perp} = -\frac{G_{E}^n}{G_{M}^n} \cdot \frac{2\sqrt{\tau(\tau+1)}\tan{\frac{\theta}{2}}}{\left(\frac{G_{E}^n}{G_{M}^n}\right)^2 + \left(\tau + 2\tau(1+\tau)\tan^2\frac{\theta}{2}\right)}
\end{equation}
    </span>

<span style="font-size:3.5em;">
\begin{equation}
A_p=P_{^3He}P_{beam}P_p A_{\perp}(Q^2)
\end{equation}
    </span>

# Load proton lookup dat

In [7]:

def load_data(file_path):
    """
    Load the data from the given file path into a pandas DataFrame.
    """
    columns = ["Q2", "GEp/GD", "dGEp/GD", "dGEp_Par/GD", "GMp/mu_p/GD", "dGMp/mu_p/GD", "dGMp_Par/mu_p/GD"]
    data = pd.read_csv(file_path, delim_whitespace=True, comment='#', names=columns)
    return data

def find_closest_row(data, input_Q2):
    """
    Find the row in the data with the Q2 value closest to the input_Q2.
    """
    closest_row = data.iloc[(data['Q2'] - input_Q2).abs().argmin()]
    return closest_row

def main(input_Q2):
    file_path = '../DB/proton_lookup.dat'
    data = load_data(file_path)
    closest_row = find_closest_row(data, input_Q2)
    return closest_row



row=main(Q2)
row

R=np.round(row[1]/row[4]/mup,4)
print(f"GEp/GMp: {R}")

GEp/GMp: 0.0964


# Main

In [70]:
#in GeV^2
m=.938
if config=="2":
    tau=Q2/(4*m**2)
    theta=29.5*pi/180
if config=="3":
    tau=Q2/(4*m**2)
    theta=36.5*pi/180
if config=="4":
    tau=Q2/(4*m**2)
    theta=35*pi/180

def Aperp(R, tau, theta):
    tan_half_theta = np.tan(np.radians(theta) / 2)
    numerator = -R * 2 * np.sqrt(tau * (tau + 1)) * tan_half_theta
    denominator = R**2 + (tau + 2 * tau * (1 + tau) * tan_half_theta**2)
    A_perp = numerator / denominator
    return A_perp

Ap=Aperp(R,tau,theta)
P_p=-.04 # seamus thesis
P_beam=CONFIG.Function_JSON("Pbeam",f"../config/cuts{config}.cfg")
P_target=CONFIG.Function_JSON("Ptarget",f"../config/cuts{config}.cfg")

Aphys=Ap*P_p*P_beam*P_target/100/100

In [73]:
Aphys

3.4069996556184517e-06

In [74]:
np.save(f'CorrectionArrays/Pass{pas}/Aproton{config}.npy',np.array(Aphys))